In [1]:
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
from nltk.util import ngrams
nltk.download('stopwords')
nltk.download('wordnet')
import unicodedata
import pyodbc

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rhale\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rhale\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 3000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 4)

In [3]:
# Function to clean up the data
def clean_string(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', ' ', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers
    # text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string

In [4]:
English = spacy.load('en_core_web_sm')
English.max_length = 3000000

In [5]:
# Get the data
# Only getting the top 50 domains for testing purposes
conn = pyodbc.connect('Driver={SQL Server};'
                           'Server=asdf;'
                           'Database=asdf;'
                           'Trusted_Connection=yes;')

sql = """
SELECT --TOP(50)
Domain, STRING_AGG(PageText, ',') AS PageText
FROM CARFScrape
GROUP BY Domain
"""

df_cat_text_by_domain = pd.read_sql(sql, conn)

C:\Users\rhale\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
# Apply the cleaning function to the PageText field to standardize, etc.
df_cat_text_by_domain['PageTextCleaned'] = df_cat_text_by_domain['PageText'].apply(lambda x: clean_string(x, stem='Lem'))
#df_cat_text_by_domain['PageTextCleaned'] = df_cat_text_by_domain['PageText']

In [7]:
# These are the keywords that we want to search for
key_words_orig = ['Metal buildings' , 'Building erection' , 'Erect metal buildings' 
, 'Structural engineering' , 'Structural design' , 'Structural fabrication' 
, 'Structural steel fabrication' , 'Structural framing construction' , 'Steel erection' 
, 'Steel building solutions' , 'Commercial' , 'Warehouse' 
, 'Industrial' , 'Aviation' , 'Agricultural' 
, 'Residential' , 'Building design' , 'Building steel structures' 
, 'Metal building design' , 'Tekla' , '3d building designer' 
, 'Metal building kit' , 'Steel building kit' , 'Pre-engineered steel building kit' 
, 'Red iron I-beam frames' , 'Roof purlins' , 'Wall girts' 
, 'Prefabricated structure']

key_words = ['3d building designer' , 'Building design' , 'Building erection' 
, 'Building steel structures' , 'Erect metal buildings' , 'Erector' 
, 'Metal building' , 'Metal building design' , 'Metal building kit' 
, 'Pre-engineered' , 'Pre-engineered steel building kit' , 'Prefabricated' 
, 'Prefabricated structure' , 'Purlin' , 'Red iron I-beam frames' 
, 'Roof purlins' , 'Steel building' , 'Steel building kit' 
, 'Steel building solutions' , 'Steel Erection' , 'Structural design' 
, 'Structural engineering' , 'Structural fabrication' , 'Structural framing construction' 
, 'Structural Steel Fabrication' , 'Tekla models' , 'Wall girts' 
, 'IAS 472' , 'IAS 473' , 'Associated Builder and Contractors' 
, 'Keystone Quality Assurance Program' , 'Metal Building Contractors & Erectors Association' , 'Metal Building Manufacturers Association' 
, 'Steel home' , 'CEES' , 'ZEES' , 'Z Purlin' , 'CEE' , 'ZEE' , 'Girts' , 'Z Girt' , 'Structural Sections' 
, 'Cold Formed Structural Sections' , 'Eave Struts' , 'CEE Purlin' , 'Red Oxide' , 'Bridge' , 'Highway' 
, 'Heavy steel', 'Commercial' , 'Warehouse' , 'Industrial' , 'Aviation' , 'Agricultural', 'Residential' ]

In [8]:
# Clean and standardize the keywords using the same function applied to the PageText
cleaned_key_words = []
for item in key_words:
    cleaned_key_words.append(clean_string(item, stem='Lem'))

In [9]:
#cleaned_key_words = key_words

In [10]:
# Create a new dataframe where each column is one of the keywords
data = pd.DataFrame(columns=cleaned_key_words)

In [11]:
data.columns

Index(['3d building designer', 'building design', 'building erection', 'building steel structure', 'erect metal building', 'erector', 'metal building', 'metal building design', 'metal building kit', 'preengineered', 'preengineered steel building kit', 'prefabricated', 'prefabricated structure', 'purlin', 'red iron ibeam frame', 'roof purlins', 'steel building', 'steel building kit', 'steel building solution', 'steel erection', 'structural design', 'structural engineering', 'structural fabrication', 'structural framing construction', 'structural steel fabrication', 'tekla model', 'wall girts', 'ia 472', 'ia 473', 'associated builder contractor', 'keystone quality assurance program', 'metal building contractor erectors association', 'metal building manufacturer association', 'steel home', 'cees', 'zee', 'z purlin', 'cee', 'zee', 'girts', 'z girt', 'structural section', 'cold formed structural section', 'eave strut', 'cee purlin', 'red oxide', 'bridge', 'highway', 'heavy steel', 'commerci

In [12]:
# Add the Domain and PageTextCleaned columns to this new dataframe and re-order the columns
data['Domain'] = df_cat_text_by_domain['Domain']
data['PageTextCleaned'] = df_cat_text_by_domain['PageTextCleaned']

data = data[['Domain', 'PageTextCleaned', '3d building designer', 'building design', 'building erection', 'building steel structure', 'erect metal building', 'erector', 'metal building', 'metal building design', 'metal building kit', 'preengineered', 'preengineered steel building kit', 'prefabricated', 'prefabricated structure', 'purlin', 'red iron ibeam frame', 'roof purlins', 'steel building', 'steel building kit', 'steel building solution', 'steel erection', 'structural design', 'structural engineering', 'structural fabrication', 'structural framing construction', 'structural steel fabrication', 'tekla model', 'wall girts', 'ia 472', 'ia 473', 'associated builder contractor', 'keystone quality assurance program', 'metal building contractor erectors association', 'metal building manufacturer association', 'steel home', 'cees', 'zee', 'z purlin', 'cee', 'zee', 'girts', 'z girt', 'structural section', 'cold formed structural section', 'eave strut', 'cee purlin', 'red oxide', 'bridge', 'highway', 'heavy steel', 'commercial', 'warehouse', 'industrial', 'aviation', 'agricultural', 'residential']]

In [13]:
data.head(4)

,Domain,PageTextCleaned,3d building designer,building design,building erection,building steel structure,erect metal building,erector,metal building,metal building design,metal building kit,preengineered,preengineered steel building kit,prefabricated,prefabricated structure,purlin,red iron ibeam frame,roof purlins,steel building,steel building kit,steel building solution,steel erection,structural design,structural engineering,structural fabrication,structural framing construction,structural steel fabrication,tekla model,wall girts,ia 472,ia 473,associated builder contractor,keystone quality assurance program,metal building contractor erectors association,metal building manufacturer association,steel home,cees,zee,zee,z purlin,cee,zee,zee,girts,z girt,structural section,cold formed structural section,eave strut,cee purlin,red oxide,bridge,highway,heavy steel,commercial,warehouse,industrial,aviation,agricultural,residential
0,advaluminum.com,70 capture 28 oct 2004 22 jul 2022 jan jul aug...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,agri-greenusa.com,16 capture 10 aug 2013 09 dec 2021 oct dec jan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,allstateerectors.net,61 capture 29 dec 2010 07 dec 2021 jul nov dec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bennettpe.com,19 capture 19 apr 2013 22 dec 2021 jan dec jan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#data['metal building'] = data['PageTextCleaned'].str.contains('metal building')

In [15]:
# Now we're going to loop through all of the columns.
# For each column:
#    1. For each Domain, search for the column name in the PageTextCleaned
#    2. Assign True/False to the value of that column for that domain

# First get a list of columns to loop through, exclude the domain/text
col_list =  data.columns
col_list = col_list[2:]

# Now loop through them and identify the keywords
for col_name in col_list:
    data[col_name] = data['PageTextCleaned'].str.contains(col_name)

In [16]:
data.head(10)

,Domain,PageTextCleaned,3d building designer,building design,building erection,building steel structure,erect metal building,erector,metal building,metal building design,metal building kit,preengineered,preengineered steel building kit,prefabricated,prefabricated structure,purlin,red iron ibeam frame,roof purlins,steel building,steel building kit,steel building solution,steel erection,structural design,structural engineering,structural fabrication,structural framing construction,structural steel fabrication,tekla model,wall girts,ia 472,ia 473,associated builder contractor,keystone quality assurance program,metal building contractor erectors association,metal building manufacturer association,steel home,cees,zee,zee,z purlin,cee,zee,zee,girts,z girt,structural section,cold formed structural section,eave strut,cee purlin,red oxide,bridge,highway,heavy steel,commercial,warehouse,industrial,aviation,agricultural,residential
0,advaluminum.com,70 capture 28 oct 2004 22 jul 2022 jan jul aug...,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,True,True,True,True,False,False,False,True,False,False,False,False,False,True,True,True,False,True,True
1,agri-greenusa.com,16 capture 10 aug 2013 09 dec 2021 oct dec jan...,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,True
2,allstateerectors.net,61 capture 29 dec 2010 07 dec 2021 jul nov dec...,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,bennettpe.com,19 capture 19 apr 2013 22 dec 2021 jan dec jan...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,bethel-eng.com,due planned power outage friday 114 8am1pm pst...,False,True,False,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,True
5,bmpmetals.net,69 capture 19 may 2001 13 mar 2022 jan mar apr...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True
6,brandonbell.com,2 capture 30 oct 2016 04 mar 2021 oct mar apr ...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True
7,cailor.com,15 capture 01 may 2011 23 dec 2021 sep dec jan...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,

In [17]:
# Wide format is not too useful for doing analytics, convert to long format using melt
data_melted = data.melt(id_vars=['Domain', 'PageTextCleaned'], 
                        value_vars=col_list, 
                        value_name='KeywordOutcome', 
                        var_name='Keyword')

In [18]:
data_melted[['Domain', 'Keyword', 'KeywordOutcome']].to_csv('Quicken_Keywords.csv')

In [19]:
# Now for each domain we can get a count of keywords
data_melted_group = data_melted.groupby(['Domain','Keyword', 'KeywordOutcome'], as_index = False)\
     .agg(Count = ('Domain', 'count'))

In [20]:
# Most common keywords
# For Keyword=True, sum up how many website have that keyword
# Then calculate the percent that had the keyword
common_keywords = data_melted_group[data_melted_group['KeywordOutcome']==True]\
                                .groupby(['Keyword'], as_index=False)\
                                .agg(TotalTrue=('Count','sum'))
common_keywords['Percent'] = common_keywords['TotalTrue']/len(data)

In [21]:
domain_keywords = data_melted_group[data_melted_group['KeywordOutcome']==True]\
                                .groupby(['Domain'], as_index=False)\
                                .agg(TotalTrue=('Count','sum'))

In [22]:
domain_keywords.sort_values(by='TotalTrue', ascending=False)

,Domain,TotalTrue
31,americanbuildings.com,26
289,kirbybuildingsystems.com,24
8,acibuildingsystems.com,23
415,reedsmetals.com,23
170,edtengineers.com,22
537,varcopruden.com,22
142,curvcosteelbuildings.com,19
32,americansteelspan.com,19
166,dunnbuildingcompany.com,18
500,structuresintl.com,18


In [23]:
common_keywords.sort_values(by='TotalTrue', ascending=False)

,Keyword,TotalTrue,Percent
10,commercial,414,0.6198
17,industrial,327,0.4895
27,residential,312,0.4671
7,cee,304,0.4551
3,bridge,202,0.3024
41,warehouse,175,0.2620
35,structural engineering,159,0.2380
4,building design,145,0.2171
34,structural design,129,0.1931
18,metal building,118,0.1766
